<a href="https://colab.research.google.com/github/jpgarciaortiz/DWT-ANN/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous steps


Download MRVC code from GitHub

In [67]:
!rm -rf MRVC ; git clone https://github.com/jpgarciaortiz/MRVC.git

Add the MRVC to the source path

In [2]:
import sys
sys.path.append("MRVC/src")

Install additional packages

In [3]:
!pip install colored

     |████████████████████████████████| 56 kB 3.2 MB/s 
  Created wheel for colored: filename=colored-1.4.2-py3-none-any.whl size=14023 sha256=00ddf0828269d7fc1af652286d684b3b68098932eccd20af928ac0442f37c573
  Stored in directory: /root/.cache/pip/wheels/5f/e1/fb/d0e85a8383ff58962319bb81c46e398fa1f4bb9e1feb0f81c4
Successfully built colored


In [66]:
!pip install ISR

In [5]:
!pip install 'h5py<3.0.0'

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


Import packages

In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
import image_3 as image
import image_1 as component
import DWT
import pywt
import distortion
import YCoCg as YUV
import deadzone as Q
import math
import information
import numpy as np
from PIL import Image

# Test code

Function to write DWT decomposition to PNG

In [41]:
def write_compact_decomposition16(decom, prefix, image_number):
    rows = decom[len(decom)-1][0].shape[0]*2
    cols = decom[len(decom)-1][0].shape[1]*2
    coms = decom[0].shape[2]
    image_shape = (rows, cols, coms)
    view = np.empty(image_shape, np.uint16)

    # LL subband
    view[0:decom[0].shape[0],
         0:decom[0].shape[1]] = (decom[0].astype(np.int32) + 32768).astype(np.uint16)

    for l in range(len(decom)-1):

        # LH
        view[0:decom[l+1][0].shape[0],
             decom[l+1][0].shape[1]:decom[l+1][0].shape[1]*2] =\
                (decom[l+1][0].astype(np.int32) + 32768).astype(np.uint16)

        # HL
        view[decom[l+1][1].shape[0]:decom[l+1][1].shape[0]*2,
             0:decom[l+1][1].shape[1]] =\
                (decom[l+1][1].astype(np.int32) + 32768).astype(np.uint16)

        # HH
        view[decom[l+1][2].shape[0]:decom[l+1][2].shape[0]*2,
             decom[l+1][2].shape[1]:decom[l+1][2].shape[1]*2] =\
                (decom[l+1][2].astype(np.int32) + 32768).astype(np.uint16)
            
    return image.write(view, prefix, image_number)
    

def write_compact_decomposition(decom, prefix, image_number):
    rows = decom[len(decom)-1][0].shape[0]*2
    cols = decom[len(decom)-1][0].shape[1]*2
    coms = decom[0].shape[2]
    image_shape = (rows, cols, coms)
    view = np.empty(image_shape, np.uint8)

    # LL subband
    view[0:decom[0].shape[0],
         0:decom[0].shape[1]] = (decom[0].astype(np.int16) + 128).astype(np.uint16)

    for l in range(len(decom)-1):

        # LH
        view[0:decom[l+1][0].shape[0],
             decom[l+1][0].shape[1]:decom[l+1][0].shape[1]*2] =\
                (decom[l+1][0].astype(np.int16) + 128).astype(np.uint16)

        # HL
        view[decom[l+1][1].shape[0]:decom[l+1][1].shape[0]*2,
             0:decom[l+1][1].shape[1]] =\
                (decom[l+1][1].astype(np.int16) + 128).astype(np.uint16)

        # HH
        view[decom[l+1][2].shape[0]:decom[l+1][2].shape[0]*2,
             decom[l+1][2].shape[1]:decom[l+1][2].shape[1]*2] =\
                (decom[l+1][2].astype(np.int16) + 128).astype(np.uint16)
            
    return image.write(view, prefix, image_number)

Read the image

In [45]:
x = image.read('MRVC/sequences/lena_color/')
#Image.fromarray(x)

Wavelet parameters

In [68]:
N_levels = 1
wavelet = pywt.Wavelet("Haar")
#wavelet = pywt.Wavelet("db1")
#wavelet = pywt.Wavelet("db5")
#wavelet = pywt.Wavelet("db15")
#wavelet = pywt.Wavelet("bior3.1")
#wavelet = pywt.Wavelet("bior3.3")
#wavelet = pywt.Wavelet("bior5.5")

In [69]:
y = DWT.analyze(x, wavelet, N_levels)
#z = DWT.synthesize(y, wavelet, N_levels).astype(np.uint8)

In [70]:
len0 = write_compact_decomposition(y, 'test', 0)
print(f'File length 0: {len0} bytes')

File length 0: 531583 bytes


In [71]:
ll = y[0]
lh = y[1][0]
hl = y[1][1]
hh = y[1][2]

In [72]:
ll_i = (image.normalize(ll) * 255.0).astype('uint8')
#Image.fromarray(ll_i)

In [73]:
from ISR.models import RDN

rdn = RDN(weights='psnr-small')
ll_i2 = rdn.predict(ll_i)
#Image.fromarray(ll_i2)

In [74]:
yp = DWT.analyze(ll_i2, wavelet, N_levels)

In [75]:
lhp = yp[1][0]
hlp = yp[1][1]
hhp = yp[1][2]

In [76]:
yp[1] = (lh - lhp, hl - hlp, hh - hhp)


In [77]:
len1 = write_compact_decomposition(yp, 'test', 1)
print(f'File length 1: {len1} bytes')

File length 1: 529213 bytes


In [78]:
dif = len0 - len1
gain = (dif * 100) / len0
print(f'Compression gain: {gain:.2}%')

Compression gain: 0.45%
